In [1]:
suppressWarnings(suppressPackageStartupMessages({
    library(tidyverse)
    library(data.table)
}))


Here, we aggregate the PRS scores computed on UK Biobank individuals into one file

The resulting file has `FID` and `IID` (as the indentifier for individuals) as well as PRS score columns for each model.

All models are derived by applying LD clumping on HGI meta-analyzed summary statistics


In [2]:
source('0_parameters.sh')


In [3]:
HGI_case_controls <- c('A2', 'B1', 'B2', 'C2')
HGI_suffices <- c('leave_23andme', 'leave_UKBB_23andme', 'eur_leave_23andme', 'eur_leave_ukbb_23andme')
clump_p1s <- c('1e-3', '1e-4', '1e-5')


In [4]:
HGI_case_control <- 'B2'
HGI_suffix <- 'eur_leave_ukbb_23andme'
clump_p1 <- '1e-5'

In [18]:
lapply(HGI_suffices, function(HGI_suffix){
lapply(HGI_case_controls, function(HGI_case_control){
lapply(clump_p1s, function(clump_p1){

    file.path(data_d, 'UKB_PRS_clump', UKB_cal_f) %>%
    str_replace('.tsv.gz$', paste0('.clump', clump_p1, '.sscore.zst')) %>%
    str_replace('@@HGI_case_control@@', HGI_case_control) %>%
    str_replace('@@HGI_suffix@@', HGI_suffix) -> PRS_f

    fread(
        cmd=paste('zstdcat', PRS_f),
        colClasses = c('#FID'='character', 'IID'='character'),
        select=c('#FID', 'IID', 'SCORE1_SUM')
    ) %>%
    rename('FID'='#FID') %>%
    rename(!!paste('PRS', HGI_case_control, HGI_suffix, clump_p1, sep='.') := 'SCORE1_SUM')
    
}) %>% reduce(function(x, y){inner_join(x, y, by=c('FID', 'IID'))})
}) %>% reduce(function(x, y){inner_join(x, y, by=c('FID', 'IID'))})
}) %>% reduce(function(x, y){inner_join(x, y, by=c('FID', 'IID'))}) -> PRS_df


In [24]:
PRS_df %>% colnames()


[1] "FID"                                "IID"                               
 [3] "PRS.A2.leave_23andme.1e-3"          "PRS.A2.leave_23andme.1e-4"         
 [5] "PRS.A2.leave_23andme.1e-5"          "PRS.B1.leave_23andme.1e-3"         
 [7] "PRS.B1.leave_23andme.1e-4"          "PRS.B1.leave_23andme.1e-5"         
 [9] "PRS.B2.leave_23andme.1e-3"          "PRS.B2.leave_23andme.1e-4"         
[11] "PRS.B2.leave_23andme.1e-5"          "PRS.C2.leave_23andme.1e-3"         
[13] "PRS.C2.leave_23andme.1e-4"          "PRS.C2.leave_23andme.1e-5"         
[15] "PRS.A2.leave_UKBB_23andme.1e-3"     "PRS.A2.leave_UKBB_23andme.1e-4"    
[17] "PRS.A2.leave_UKBB_23andme.1e-5"     "PRS.B1.leave_UKBB_23andme.1e-3"    
[19] "PRS.B1.leave_UKBB_23andme.1e-4"     "PRS.B1.leave_UKBB_23andme.1e-5"    
[21] "PRS.B2.leave_UKBB_23andme.1e-3"     "PRS.B2.leave_UKBB_23andme.1e-4"    
[23] "PRS.B2.leave_UKBB_23andme.1e-5"     "PRS.C2.leave_UKBB_23andme.1e-3"    
[25] "PRS.C2.leave_UKBB_23andme.1e-4"     "PRS.C2.leave_UKBB_23andme.1e-5"    
[27] "PRS.A2.eur_leave_23andme.1e-3"      "PRS.A2.eur_leave_23andme.1e-4"     
[29] "PRS.A2.eur_leave_23andme.1e-5"      "PRS.B1.eur_leave_23andme.1e-3"     
[31] "PRS.B1.eur_leave_23andme.1e-4"      "PRS.B1.eur_leave_23andme.1e-5"     
[33] "PRS.B2.eur_leave_23andme.1e-3"      "PRS.B2.eur_leave_23andme.1e-4"     
[35] "PRS.B2.eur_leave_23andme.1e-5"      "PRS.C2.eur_leave_23andme.1e-3"     
[37] "PRS.C2.eur_leave_23andme.1e-4"      "PRS.C2.eur_leave_23andme.1e-5"     
[39] "PRS.A2.eur_leave_ukbb_23andme.1e-3" "PRS.A2.eur_leave_ukbb_23andme.1e-4"
[41] "PRS.A2.eur_leave_ukbb_23andme.1e-5" "PRS.B1.eur_leave_ukbb_23andme.1e-3"
[43] "PRS.B1.eur_leave_ukbb_23andme.1e-4" "PRS.B1.eur_leave_ukbb_23andme.1e-5"
[45] "PRS.B2.eur_leave_ukbb_23andme.1e-3" "PRS.B2.eur_leave_ukbb_23andme.1e-4"
[47] "PRS.B2.eur_leave_ukbb_23andme.1e-5" "PRS.C2.eur_leave_ukbb_23andme.1e-3"
[49] "PRS.C2.eur_leave_ukbb_23andme.1e-4" "PRS.C2.eur_leave_ukbb_23andme.1e-5"

In [25]:
PRS_df %>%
rename('#FID' = 'FID') %>%
fwrite(file.path(data_d, 'UKB_PRS_clump.phe'), sep='\t', na = "NA", quote=F)
